# Setup #

In [ ]:
# import sys
# !cp ../input/rapids/rapids.21.06 /opt/conda/envs/rapids.tar.gz
# !cd /opt/conda/envs/ && tar -xzvf rapids.tar.gz > /dev/null
# sys.path = ["/opt/conda/envs/rapids/lib/python3.7/site-packages"] + sys.path
# sys.path = ["/opt/conda/envs/rapids/lib/python3.7"] + sys.path
# sys.path = ["/opt/conda/envs/rapids/lib"] + sys.path 
# !cp /opt/conda/envs/rapids/lib/libxgboost.so /opt/conda/lib/

In [ ]:
import os
import sys
import shutil
from glob import glob
import multiprocessing as mp
import gc
from pathlib import Path
from scipy import stats
from scipy.special import boxcox, softmax
from scipy import sparse

from multiprocessing import cpu_count
import copy
import pickle
import warnings
from datetime import datetime, timedelta
from time import time, sleep, mktime
from matplotlib import font_manager as fm, rc, rcParams
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import re
import random as rnd
import psutil
from optuna import Trial, create_study
from optuna.samplers import TPESampler

import numpy as np
from numpy import array, nan, random as np_rnd, where
import pandas as pd
from pandas import DataFrame as dataframe, Series as series, isna, read_csv
from pandas.tseries.offsets import DateOffset

from sklearn.model_selection import train_test_split as tts, StratifiedKFold, StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler, RobustScaler, KBinsDiscretizer
from sklearn import metrics
from sklearn.compose import ColumnTransformer
# config_missingpy(); from missingpy import MissForest
from sklearn.impute import KNNImputer
from optuna import Trial, create_study
from sklearn.model_selection import GroupKFold, GroupShuffleSplit, StratifiedGroupKFold
from sklearn.feature_extraction.text import TfidfVectorizer

try:
    import cudf as cd
    import cupy as cp
    from cuml.cluster import KMeans
    from cuml.neighbors import NearestNeighbors
    from cuml.metrics.cluster import silhouette_score
except:
    print("RAPIDS Import ERROR")

import catboost as cat
import xgboost as xgb

# # ===== tensorflow =====
# import tensorflow as tf
# from tensorflow import random as tf_rnd
# from tensorflow.keras.models import Model
# from tensorflow.keras.models import Sequential
# from tensorflow.keras import layers
# from tensorflow.keras import activations
# from tensorflow.keras import optimizers
# from tensorflow.keras import metrics as tf_metrics
# from tensorflow.keras import callbacks as tf_callbacks
# from tqdm.keras import TqdmCallback
# import tensorflow_addons as tfa
# from tensorflow.keras.utils import plot_model
# from keras.utils.layer_utils import count_params

# import keras_tuner as kt
# from keras_tuner import HyperModel
# import tensorflow_hub as tf_hub
# import tensorflow_recommenders as tfrs

# # GPU check
# if tf.test.gpu_device_name() != '/device:GPU:0':
#     print('GPU device not found')
# else:
#     print('Found GPU')

# # GPU memory setting
# gpus = tf.config.list_physical_devices('GPU')
# if gpus:
#   try:
#     tf.config.experimental.set_memory_growth(gpus[0], True)
#   except RuntimeError as e:
#     print(e)

warnings.filterwarnings(action='ignore')
rcParams['axes.unicode_minus'] = False
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 200)
# plt.rc('font', family='NanumSquareB')

data_dir = Path('../input/AI4Code')

In [ ]:
# ===== utility functions =====
# label encoding for categorical column with excepting na value
def seed_everything(seed=42):
    # python random module
    rnd.seed(seed)
    # numpy random
    np_rnd.seed(seed)
#     # tf random
#     tf_rnd.set_seed(seed)
    # RAPIDS random
    try:
        cp.random.seed(seed)
    except:
        pass
def which(bool_list):
    return where(bool_list)[0]
def easyIO(x=None, path=None, op="r"):
    tmp = None
    if op == "r":
        with open(path, "rb") as f:
            tmp = pickle.load(f)
        return tmp
    elif op == "w":
        with open(path, "wb") as f:
            pickle.dump(x, f)
    else:
        print("Unknown operation type")
def diff(first, second):
    second = set(second)
    return [item for item in first if item not in second]
def findIdx(data_x, col_names):
    return [int(i) for i, j in enumerate(data_x) if j in col_names]
def orderElems(for_order, using_ref):
    return [i for i in using_ref if i in for_order]
# concatenate by row
def cbr(df1, df2):
    if type(df1) == series:
        tmp_concat = series(pd.concat([dataframe(df1), dataframe(df2)], axis=0, ignore_index=True).iloc[:,0])
        tmp_concat.reset_index(drop=True, inplace=True)
    elif type(df1) == dataframe:
        tmp_concat = pd.concat([df1, df2], axis=0, ignore_index=True)
        tmp_concat.reset_index(drop=True, inplace=True)
    elif type(df1) == np.ndarray:
        tmp_concat = np.concatenate([df1, df2], axis=0)
    else:
        print("Unknown Type: return 1st argument")
        tmp_concat = df1
    return tmp_concat
def change_width(ax, new_value):
    for patch in ax.patches :
        current_width = patch.get_width()
        adj_value = current_width - new_value
        # we change the bar width
        patch.set_width(new_value)
        # we recenter the bar
        patch.set_x(patch.get_x() + adj_value * .5)
def week_of_month(date):
    month = date.month
    week = 0
    while date.month == month:
        week += 1
        date -= timedelta(days=7)
    return week
def getSeason(date):
    month = date.month
    if month in [3, 4, 5]:
        return "Spring"
    elif month in [6, 7, 8]:
        return "Summer"
    elif month in [9, 10, 11]:
        return "Fall"
    else:
        return "Winter"
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print('Error: Creating directory. ' + directory)
# def softmax(x):
#     max = np.max(x, axis=1, keepdims=True)  # returns max of each row and keeps same dims
#     e_x = np.exp(x - max)  # subtracts each row with its max value
#     sum = np.sum(e_x, axis=1, keepdims=True)  # returns sum of each row and keeps same dims
#     f_x = e_x / sum
#     return f_x
def sigmoid(x):
    return 1/(1 + np.exp(-x))
def dispPerformance(result_dic):
    perf_table = dataframe()
    index_names = []
    for k, v in result_dic.items():
        index_names.append(k)
        perf_table = pd.concat([perf_table, series(v["performance"]).to_frame().T], ignore_index=True, axis=0)
    perf_table.index = index_names
    perf_table.sort_values(perf_table.columns[0], inplace=True)
    print(perf_table)
    return perf_table
def powspace(start, stop, power, num):
    start = np.power(start, 1/float(power))
    stop = np.power(stop, 1/float(power))
    return np.power(np.linspace(start, stop, num=num), power)
def xgb_custom_lossfunction(alpha = 1):
    def support_under_mse(label, pred):
        # grad : 1차 미분
        # hess : 2차 미분
        residual = (label - pred).astype("float")
        grad = np.where(residual > 0, -2 * alpha * residual, -2 * residual)
        hess = np.where(residual > 0, 2 * alpha, 2.0)
        return grad, hess
    return support_under_mse
def pd_flatten(df):
    df = df.unstack()
    df.index = [str(i) + "_" + str(j) for i, j in df.index]
    return df
def tf_losses_rmse(y_true, y_pred, sample_weight=None):
    return tf.sqrt(tf.reduce_mean((y_true - y_pred) ** 2)) if sample_weight is None else tf.sqrt(tf.reduce_mean(((y_true - y_pred) ** 2) * sample_weight))
def tf_loss_nmae(y_true, y_pred, sample_weight=False):
    mae = tf.reduce_mean(tf.math.abs(y_true - y_pred))
    score = tf.math.divide(mae, tf.reduce_mean(tf.math.abs(y_true)))
    return score
def text_extractor(string, lang="eng", spacing=True):
    # # 괄호를 포함한 괄호 안 문자 제거 정규식
    # re.sub(r'\([^)]*\)', '', remove_text)
    # # <>를 포함한 <> 안 문자 제거 정규식
    # re.sub(r'\<[^)]*\>', '', remove_text)
    if lang == "eng":
        text_finder = re.compile('[^ A-Za-z]') if spacing else re.compile('[^A-Za-z]')
    elif lang == "kor":
        text_finder = re.compile('[^ ㄱ-ㅣ가-힣+]') if spacing else re.compile('[^ㄱ-ㅣ가-힣+]')
    # default : kor + eng
    else:
        text_finder = re.compile('[^ A-Za-zㄱ-ㅣ가-힣+]') if spacing else re.compile('[^A-Za-zㄱ-ㅣ가-힣+]')
    return text_finder.sub('', string)
def memory_usage(message='debug'):
    # current process RAM usage
    p = psutil.Process()
    rss = p.memory_info().rss / 2 ** 20 # Bytes to MB
    print(f"[{message}] memory usage: {rss: 10.3f} MB")
    return rss
class MyLabelEncoder:
    def __init__(self, preset={}):
        # dic_cat format -> {"col_name": {"value": replace}}
        self.dic_cat = preset
    def fit_transform(self, data_x, col_names):
        tmp_x = copy.deepcopy(data_x)
        for i in col_names:
            # if key is not in dic, update dic
            if i not in self.dic_cat.keys():
                tmp_dic = dict.fromkeys(sorted(set(tmp_x[i]).difference([nan])))
                label_cnt = 0
                for j in tmp_dic.keys():
                    tmp_dic[j] = label_cnt
                    label_cnt += 1
                self.dic_cat[i] = tmp_dic
            # transform value which is not in dic to nan
            tmp_x[i] = tmp_x[i].astype("object")
            conv = tmp_x[i].replace(self.dic_cat[i])
            for conv_idx, j in enumerate(conv):
                if j not in self.dic_cat[i].values():
                    conv[conv_idx] = nan
            # final return
            tmp_x[i] = conv.astype("float")
        return tmp_x
    def transform(self, data_x):
        tmp_x = copy.deepcopy(data_x)
        for i in self.dic_cat.keys():
            # transform value which is not in dic to nan
            tmp_x[i] = tmp_x[i].astype("object")
            conv = tmp_x[i].replace(self.dic_cat[i])
            for conv_idx, j in enumerate(conv):
                if j not in self.dic_cat[i].values():
                    conv[conv_idx] = nan
            # final return
            tmp_x[i] = conv.astype("float")
        return tmp_x
    def clear(self):
        self.dic_cat = {}
class MyOneHotEncoder:
    def __init__(self, label_preset={}):
        self.dic_cat = {}
        self.label_preset = label_preset
    def fit_transform(self, data_x, col_names):
        tmp_x = dataframe()
        for i in data_x:
            if i not in col_names:
                tmp_x = pd.concat([tmp_x, dataframe(data_x[i])], axis=1)
            else:
                if not ((data_x[i].dtype.name == "object") or (data_x[i].dtype.name == "category")):
                    print(F"WARNING : {i} is not object or category")
                self.dic_cat[i] = OneHotEncoder(sparse=False, handle_unknown="ignore")
                conv = self.dic_cat[i].fit_transform(dataframe(data_x[i])).astype("int")
                col_list = []
                for j in self.dic_cat[i].categories_[0]:
                    if i in self.label_preset.keys():
                        for k, v in self.label_preset[i].items():
                            if v == j:
                                col_list.append(str(i) + "_" + str(k))
                    else:
                        col_list.append(str(i) + "_" + str(j))
                conv = dataframe(conv, columns=col_list)
                tmp_x = pd.concat([tmp_x, conv], axis=1)
        return tmp_x
    def transform(self, data_x):
        tmp_x = dataframe()
        for i in data_x:
            if not i in list(self.dic_cat.keys()):
                tmp_x = pd.concat([tmp_x, dataframe(data_x[i])], axis=1)
            else:
                if not ((data_x[i].dtype.name == "object") or (data_x[i].dtype.name == "category")):
                    print(F"WARNING : {i} is not object or category")
                conv = self.dic_cat[i].transform(dataframe(data_x[i])).astype("int")
                col_list = []
                for j in self.dic_cat[i].categories_[0]:
                    if i in self.label_preset.keys():
                        for k, v in self.label_preset[i].items():
                            if v == j: col_list.append(str(i) + "_" + str(k))
                    else:
                        col_list.append(str(i) + "_" + str(j))
                conv = dataframe(conv, columns=col_list)
                tmp_x = pd.concat([tmp_x, conv], axis=1)
        return tmp_x
    def clear(self):
        self.dic_cat = {}
        self.label_preset = {}
class MyKNNImputer:
    def __init__(self, k=5):
        self.imputer = KNNImputer(n_neighbors=k)
        self.dic_cat = {}
    def fit_transform(self, x, cat_vars=None):
        if cat_vars is None:
            x_imp = dataframe(self.imputer.fit_transform(x), columns=x.columns)
        else:
            naIdx = dict.fromkeys(cat_vars)
            for i in cat_vars:
                self.dic_cat[i] = diff(list(sorted(set(x[i]))), [nan])
                naIdx[i] = list(which(array(x[i].isna())))
            x_imp = dataframe(self.imputer.fit_transform(x), columns=x.columns)

            # if imputed categorical value are not in the range, adjust the value
            for i in cat_vars:
                x_imp[i] = x_imp[i].apply(lambda x: int(round(x, 0)))
                for j in naIdx[i]:
                    if x_imp[i][j] not in self.dic_cat[i]:
                        if x_imp[i][j] < self.dic_cat[i][0]:
                            x_imp[i][naIdx[i]] = self.dic_cat[i][0]
                        elif x_imp[i][j] > self.dic_cat[i][0]:
                            x_imp[i][naIdx[i]] = self.dic_cat[i][len(self.dic_cat[i]) - 1]
        return x_imp
    def transform(self, x):
        if len(self.dic_cat.keys()) == 0:
            x_imp = dataframe(self.imputer.transform(x), columns=x.columns)
        else:
            naIdx = dict.fromkeys(self.dic_cat.keys())
            for i in self.dic_cat.keys():
                naIdx[i] = list(which(array(x[i].isna())))
            x_imp = dataframe(self.imputer.transform(x), columns=x.columns)

            # if imputed categorical value are not in the range, adjust the value
            for i in self.dic_cat.keys():
                x_imp[i] = x_imp[i].apply(lambda x: int(round(x, 0)))
                for j in naIdx[i]:
                    if x_imp[i][j] not in self.dic_cat[i]:
                        if x_imp[i][j] < self.dic_cat[i][0]:
                            x_imp[i][naIdx[i]] = self.dic_cat[i][0]
                        elif x_imp[i][j] > self.dic_cat[i][0]:
                            x_imp[i][naIdx[i]] = self.dic_cat[i][len(self.dic_cat[i]) - 1]
        return x_imp
    def clear(self):
        self.imputer = None
        self.dic_cat = {}
def remove_outlier(df, std=3, mode="remove"):
    tmp_df = df.copy()
    if mode == "remove":
        outlier_mask = (np.abs(stats.zscore(tmp_df)) > std).all(axis=1)
        print("found outlier :", outlier_mask.sum())
        tmp_df = tmp_df[~outlier_mask]
    elif mode == "interpolate":
        tmp_outlier = []
        for i in tmp_df:
            outlier_mask = (np.abs(stats.zscore(tmp_df[i])) > std)
            tmp_outlier.append(outlier_mask.sum())
            if tmp_outlier[-1] == 0:
                continue
            tmp_df[i][outlier_mask] = np.nan
            tmp_df[i] = tmp_df[i].interpolate(method='linear').bfill()
        print("found outlier :", np.sum(outlier_mask))
    return tmp_df

seed_everything()

# Load Data #

In [ ]:
# define function reading the train data
def read_notebook(path):
    return (
        pd.read_json(
            path,
            dtype={'cell_type': 'category', 'source': 'str'})
        .assign(id=path.stem)
        .rename_axis('cell_id')
    )
def update_rawdata_dic(x, proc_id):
    rawdata = [read_notebook(path) for path in x]
    tmp_dic.update({proc_id: rawdata})
    print("job finished :", proc_id, "\n")

In [ ]:
# read one by one alogn with the notebook ids
# random select train dataset
paths_train = list((data_dir / 'train').glob('*.json'))
np_rnd.seed(1)
paths_train = list(array(paths_train)[np_rnd.randint(0, len(paths_train)-1, size=10000, dtype=int)])
# paths_train = list(array(paths_train)[np_rnd.randint(0, len(paths_train)-1, size=100, dtype=int)])

# ===== original code =====
start_time = time()
notebooks_train = [
    read_notebook(path) for path in tqdm(paths_train, desc='Train NBs')
]
print(time() - start_time)

In [ ]:
# concatenate the data by row
df_full_raw = (
    pd.concat(notebooks_train)
    .set_index('id', append=True)
    .swaplevel()
    .sort_index(level='id', sort_remaining=False)
)
print(df_full_raw)

In [ ]:
paths_test = list((data_dir / 'test').glob('*.json'))
notebooks_test = [
    read_notebook(path) for path in tqdm(paths_test, desc='Test NBs')
]

In [ ]:
df_test = (
    pd.concat(notebooks_test)
    .set_index('id', append=True)
    .swaplevel()
    .sort_index(level='id', sort_remaining=False)
)

In [ ]:
del notebooks_train, notebooks_test, paths_train, paths_test

# Ordering the Cells #

In [ ]:
def get_ranks(orderd, unordered, normalize=False, score_by_position=True):
    if score_by_position:
        scores = pd.Series([orderd.index(d) for d in unordered]) + 1
        return (scores / scores.max()).tolist() if normalize else scores.tolist()
    else:
        scores = pd.Series([len(orderd)-1 - orderd.index(d) for d in unordered]) + 1
        return (scores / scores.max()).tolist() if normalize else scores.tolist()

In [ ]:
df_orders = pd.read_csv(
    data_dir / 'train_orders.csv',
    index_col='id',
    squeeze=True,
).str.split()  # Split the string representation of cell_ids into a list

print(df_orders)

In [ ]:
# load the data including correct order
# join the unordered raw train data to the correctly ordered data
df_ranks = df_orders.to_frame().join(
    df_full_raw.reset_index('cell_id').groupby('id')['cell_id'].apply(list), how='right',
)
# columns order : NB_id, correct cell order, incorrect cell order
print(df_ranks.head())

In [ ]:
# get the correct rank score and save to the dictionary
ranks = {}
for id_, cell_order, cell_id in df_ranks.itertuples():
    ranks[id_] = {'cell_id': cell_id, 'rank': get_ranks(cell_order, cell_id, normalize=True, score_by_position=True)}

In [ ]:
df_full = (
    # create dataframe from the dictionary including rank score   
    pd.DataFrame.from_dict(ranks, orient='index')
    # rename the index
    .rename_axis('id')
    # flatten to the row from the list-typed correctly ordered cell_id object 
    .apply(pd.Series.explode)
    # set 2nd index as cell_id
    .set_index('cell_id', append=True)
# join the unordered dataframe to the ordered dataframe
).join(df_full_raw, on=["id", "cell_id"], how="right")

df_full = df_full.sort_values(["id", "rank"], ascending=[True, False])

In [ ]:
def restruct_rank_values(x):
    input_df = x.copy()
    for i in input_df.index.get_level_values(0).unique():
        tmp_df = input_df.loc[i].copy()
        tmp_md = []
        last_code_rank_value = 0
        interval_rank_value = tmp_df.loc[(tmp_df["cell_type"] == "code"), "rank"].diff().min()
        for idx, value in enumerate(tmp_df["cell_type"]):
            if value == "markdown":
                tmp_md.append(idx)
            elif value == "code":
                if len(tmp_md) > 0:
                    tmp_df["rank"].iloc[tmp_md] = np.linspace(last_code_rank_value, tmp_df["rank"].iloc[idx], len(tmp_md)+2)[1:-1]
                    tmp_md = []
                else:
                    pass
                last_code_rank_value = tmp_df["rank"].iloc[idx]
        # if markdown is last cell
        if len(tmp_md) > 0:
            tmp_df["rank"].iloc[tmp_md] = last_code_rank_value + interval_rank_value
            tmp_md = []
        input_df.loc[i] = tmp_df.values
    return input_df

In [ ]:
df_full.head(20)

In [ ]:
df_full["rank"] = df_full.groupby(["id", "cell_type"]).cumcount()
df_full["rank"] = df_full.groupby(["id", "cell_type"])["rank"].rank(pct=True) * 100

In [ ]:
df_full.loc[(df_full["cell_type"] == "markdown"), "rank"] = 0

In [ ]:
# get positional target value on markdown cell
df_full = restruct_rank_values(df_full)

In [ ]:
df_full = df_full.sort_values(["id", "rank"], ascending=[True, False])

In [ ]:
df_full.info()

In [ ]:
df_full.head(20)

In [ ]:
df_full.isna().sum()

In [ ]:
print(df_full["rank"].min())
print(df_full["rank"].max())

In [ ]:
del df_ranks, df_full_raw, ranks

In [ ]:
df_test["rank"] = df_test.groupby(["id", "cell_type"]).cumcount()
df_test["rank"] = df_test.groupby(["id", "cell_type"]).apply(lambda x: x["rank"][::-1]).values
df_test["rank"] = df_test.groupby(["id", "cell_type"])["rank"].rank(pct=True) * 100

In [ ]:
df_test.loc[(df_test["cell_type"] == "markdown"), "rank"] = 0

In [ ]:
df_test = df_test.sort_values(["id", "cell_type", "rank"], ascending=[True, True, False])

In [ ]:
# # slightly minus a value from the max value on test datset
# df_test.loc[df_test.groupby(["id"]).idxmax()["rank"], "rank"] -= (df_test.loc[df_test["cell_type"] == "code", "rank"].groupby("id").apply(lambda x: x.nlargest(1)[-1] - x.nlargest(2)[-1]) / 10).values

In [ ]:
df_test = df_test[["rank", "cell_type", "source"]]

In [ ]:
df_test.head(20)

In [ ]:
df_test.info()

In [ ]:
df_test.isna().sum()

In [ ]:
print(df_test["rank"].min())
print(df_test["rank"].max())

# Preprocessing & Text Cleansing

In [ ]:
df_full["cell_type"] = df_full["cell_type"].apply(lambda x: 1.0 if x == "markdown" else 0.0)

In [ ]:
df_full["cell_type"] = df_full["cell_type"].astype("float32")
df_full["rank"] = df_full["rank"].astype("float32")

In [ ]:
df_full.head()

In [ ]:
df_test["cell_type"] = df_test["cell_type"].apply(lambda x: 1.0 if x == "markdown" else 0.0)

In [ ]:
df_test["cell_type"] = df_test["cell_type"].astype("float32")
df_test["rank"] = df_test["rank"].astype("float32")

In [ ]:
df_test.head()

In [ ]:
def str_cleansing(x):
    preproc_lines = []
    for line in x.split("\n"):
        line = line.replace('\n','')
        line = line.replace('\t',' ')
        
        # remove alias names
        tmp = []
        for idx, value in enumerate(line.split()):
            if value == "as":
                tmp.append(idx)
                try:
                    tmp.append(idx + 1)
                except:
                    pass
        line = " ".join(series(line.split()).iloc[diff(list(range(len(line.split()))), tmp)].tolist())

        # replace 'import numpy as np' -> 'import numpy' to be easily tokenized
        # but, below codes are not much effective for codebert tokenizer (codebert seems not to be able to interpret 'import ~') 
        tmp = []
        skip_flag = True
        for idx, value in enumerate(line.split()):
            if skip_flag:
                if value == "import":
                    try:
                        tmp.append("import " + line.split()[idx+1])
                        skip_flag=False
                    except:
                        tmp.append("import ")
                elif value == "from":
                    try:
                        tmp.append("from " + line.split()[idx+1])
                        skip_flag=False
                    except:
                        tmp.append("from ") 
                else:
                    tmp.append(value)
            else:
                skip_flag = True
                continue
        line = " ".join(tmp)
        
        preproc_lines.append(line)
        text_finder = re.compile('[^A-Za-z]')
        preproc_lines[-1] = " ".join(text_finder.sub(' ', preproc_lines[-1]).split()).lower()
    preprocessed_script = ' '.join(preproc_lines)
    preprocessed_script = ' '.join(preprocessed_script.split())
    return preprocessed_script

In [ ]:
df_full.groupby("id").size().describe()

In [ ]:
# adjust the cell length
MIN_CELL_LEN = 8
MAX_CELL_LEN = 256
df_full = df_full.loc[df_full.groupby("id").size().index[(df_full.groupby("id").size() >= MIN_CELL_LEN) & (df_full.groupby("id").size() <= MAX_CELL_LEN)]]

# select the train data which has at least 1 cell for both code & markdown
df_full = df_full.loc[df_full.index.get_level_values(0).unique()[df_full.groupby("id")["cell_type"].apply(lambda x: True if ((x==0.0).sum() > 1) & ((x==1.0).sum() > 1) else False)]]

In [ ]:
df_full.groupby("id").size().describe()

In [ ]:
# df_full["source"][df_full["cell_type"] == 0.0] = df_full["source"][df_full["cell_type"] == 0.0].apply(lambda x: str_cleansing(x, 0))
# df_full["source"][df_full["cell_type"] == 1.0] = df_full["source"][df_full["cell_type"] == 1.0].apply(lambda x: str_cleansing(x, 1))

df_full["source"] = df_full["source"].apply(lambda x: str_cleansing(x))

In [ ]:
df_full.head(20)

In [ ]:
# df_test["source"][df_test["cell_type"] == 0.0] = df_test["source"][df_test["cell_type"] == 0.0].apply(lambda x: str_cleansing(x, 0))
# df_test["source"][df_test["cell_type"] == 1.0] = df_test["source"][df_test["cell_type"] == 1.0].apply(lambda x: str_cleansing(x, 1))

df_test["source"] = df_test["source"].apply(lambda x: str_cleansing(x))

In [ ]:
df_test.head(20)

# Training - Distilbert embedding & CatBoost Ranker

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
from torch.utils.data import DataLoader
torch_device = torch.device('cuda')

codebert_tokenizer_path = "../input/ai4code-ms-codebert/microsoft-codebert-base/tokenizer/"
codebert_model_path = "../input/ai4code-ms-codebert/microsoft-codebert-base/model/"

bert_tokenizer_path = "../input/ai4code-bert/bert-base-uncased/tokenizer/"
bert_model_path = "../input/ai4code-bert/bert-base-uncased/model/"

distilbert_tokenizer_path = "../input/ai4code-distilbert/distilbert-base-uncased/tokenizer/"
distilbert_model_path = "../input/ai4code-distilbert/distilbert-base-uncased/model/"

MAX_LEN = 256

def get_fv(tokenizer_path, model_path, x, batch_size=32, device_type="gpu"):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
    if device_type == "gpu":
        model = AutoModel.from_pretrained(model_path).to(torch_device)
    else:
        model = AutoModel.from_pretrained(model_path)
    tmp = []
    dataloader = DataLoader(x, batch_size, shuffle=False)
    for idx, value in enumerate(tqdm(dataloader)):  
        inputs = tokenizer.batch_encode_plus(
            value,
            add_special_tokens=True,
            max_length=MAX_LEN,
            padding="max_length",
            return_token_type_ids=False,
            truncation=True
        )
        if device_type == "gpu":
            input_ids = torch.tensor(inputs['input_ids']).to(torch_device)
        else:
            input_ids = torch.tensor(inputs['input_ids'])
        outputs = model(input_ids).last_hidden_state.detach().cpu().numpy().mean(axis=1)
        tmp.append(outputs)
    return np.concatenate(tmp, axis=0)

In [ ]:
df_full_fv = np.zeros((df_full.shape[0], 768))
df_full_fv = get_fv(distilbert_tokenizer_path, distilbert_model_path, df_full["source"], batch_size=32, device_type="gpu").astype("float32")
torch.cuda.empty_cache()
df_full.drop(["source"], axis=1, inplace=True)

df_test_fv = np.zeros((df_test.shape[0], 768))
df_test_fv = get_fv(distilbert_tokenizer_path, distilbert_model_path, df_test["source"], batch_size=32, device_type="gpu").astype("float32")
torch.cuda.empty_cache()
df_test.drop(["source"], axis=1, inplace=True)

In [ ]:
from bisect import bisect

def create_dataset(x, y, batch_size, shuffle=True, sample_weight=None):
    if sample_weight is not None:
        dataset = tf.data.Dataset.from_tensor_slices((x, y, sample_weight))
    else:
        dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.shuffle(int(batch_size * 4), reshuffle_each_iteration=True) if shuffle else dataset
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(4)
    return dataset
def count_inversions(a):
    inversions = 0
    sorted_so_far = []
    for i, u in enumerate(a):
        j = bisect(sorted_so_far, u)
        inversions += i - j
        sorted_so_far.insert(j, u)
    return inversions
def kendall_tau(ground_truth, predictions):
    total_inversions = 0
    total_2max = 0  # twice the maximum possible inversions across all instances
    for gt, pred in zip(ground_truth, predictions):
        ranks = [gt.index(x) for x in pred]  # rank predicted order in terms of ground truth
        total_inversions += count_inversions(ranks)
        n = len(gt)
        total_2max += n * (n - 1)
    return 1 - 4 * total_inversions / total_2max
def convert_rank_ids(x):
    # Convert the cell_id index into a column.
    x = x.reset_index('cell_id')
    # Group the cell_ids for each notebook into a list.
    x = x.groupby('id')['cell_id'].apply(list)
    return x
def flatten_list(xss):
    return [x for xs in xss for x in xs]

In [ ]:
# optuna function
def optuna_objective_function(trial: Trial, fold, train_x, train_y, train_groups, val_x, val_y, val_groups, categoIdx,
                              model_name, output_container, ntrees=1000, eta=1e-2, best_ntrees=[None]):
    if model_name == "LGB_GOSS":
        # objective
        # regession : "mae", "mse"
        # classification - binary : "binary"
        # classification - binary : "multiclass" (num_class=n)
        # ranking : "xe_ndcg_mart"

        # metric
        # regession : "mae", "mse", "rmse"
        # classification - binary : "binary_logloss", "binary_error", "auc"
        # classification - muticlass : "multi_logloss", "multi_error"
        # ranking : "ndcg", "map"

        tuning_params = {
            "learning_rate": trial.suggest_categorical("learning_rate", [1e-2, 5e-3, 1e-3]),
            "num_leaves": trial.suggest_categorical("num_leaves", [pow(2, i) - 1 for i in [4, 5, 6, 7, 8]]),
            # goss sampling hyper-parameter replacing the "sumample"
            "top_rate": trial.suggest_float("top_rate", 0.2, 0.5, step=0.1),
            "other_rate": trial.suggest_float("other_rate", 0.2, 0.5, step=0.1),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 0.8, step=0.1),
            "reg_lambda": trial.suggest_categorical("reg_lambda", list(np.linspace(1e-3, 1.0, num=150, endpoint=False)) + list(np.linspace(1.0, 1e+2, num=50, endpoint=True))),
            "min_child_weight": trial.suggest_categorical("min_child_weight", list(np.linspace(1e-3, 1.0, num=150, endpoint=False)) + list(np.linspace(1.0, 1e+2, num=50, endpoint=True))),
            "min_child_samples": trial.suggest_int("min_child_samples", 1, 51, step=2),
            "min_gain_to_split": trial.suggest_categorical("min_gain_to_split", list(np.linspace(1e-3, 1.0, num=150, endpoint=False)) + list(np.linspace(1.0, 1e+2, num=50, endpoint=True))),
            # for binary
            "scale_pos_weight": trial.suggest_categorical("scale_pos_weight", list(np.linspace(1e-3, 1.0, num=150, endpoint=False)) + list(np.linspace(1.0, 1e+2, num=50, endpoint=True))),
        }
        model = lgb.LGBMClassifier(boosting_type="goss", objective="binary",
                                   n_estimators=ntrees, device_type="gpu",
                                   random_state=fold, verbose=-1, **tuning_params)
        cb_list = [
            lgb.early_stopping(stopping_rounds=int(ntrees * 0.2), first_metric_only=True, verbose=False, min_delta=0.001),
        ]
        model.fit(train_x, train_y, categorical_feature=categoIdx,
                  eval_set=(val_x,val_y), eval_metric="auc", callbacks=cb_list)
        best_ntrees[0] = model.best_iteration_
    elif model_name == "LGB_GBM":
        # objective
        # regession : "mae", "mse"
        # classification - binary : "binary"
        # classification - binary : "multiclass" (num_class=n)
        # ranking : "xe_ndcg_mart"

        # metric
        # regession : "mae", "mse", "rmse"
        # classification - binary : "binary_logloss", "binary_error", "auc"
        # classification - muticlass : "multi_logloss", "multi_error"
        # ranking : "ndcg", "map"

        tuning_params = {
            "learning_rate": trial.suggest_categorical("learning_rate", [1e-2, 5e-3, 1e-3]),
            "num_leaves": trial.suggest_categorical("num_leaves", [pow(2, i) - 1 for i in [4, 5, 6, 7, 8]]),
            "subsample": trial.suggest_float("subsample", 0.5, 0.8, step=0.1),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 0.8, step=0.1),
            "reg_lambda": trial.suggest_categorical("reg_lambda", list(np.linspace(1e-3, 1.0, num=150, endpoint=False)) + list(np.linspace(1.0, 1e+2, num=50, endpoint=True))),
            "min_child_weight": trial.suggest_categorical("min_child_weight", list(np.linspace(1e-3, 1.0, num=150, endpoint=False)) + list(np.linspace(1.0, 1e+2, num=50, endpoint=True))),
            "min_child_samples": trial.suggest_int("min_child_samples", 1, 51, step=2),
            "min_gain_to_split": trial.suggest_categorical("min_gain_to_split", list(np.linspace(1e-3, 1.0, num=150, endpoint=False)) + list(np.linspace(1.0, 1e+2, num=50, endpoint=True))),
            # for binary
            "scale_pos_weight": trial.suggest_categorical("scale_pos_weight", list(np.linspace(1e-3, 1.0, num=150, endpoint=False)) + list(np.linspace(1.0, 1e+2, num=50, endpoint=True))),
        }

        model = lgb.LGBMClassifier(boosting_type="gbdt", objective="binary",
                                   n_estimators=ntrees, device_type="gpu",
                                   random_state=fold, verbose=-1, **tuning_params)
        cb_list = [
            lgb.early_stopping(stopping_rounds=int(ntrees * 0.2), first_metric_only=True, verbose=False, min_delta=0.001),
        ]
        model.fit(train_x, train_y, categorical_feature=categoIdx,
                  eval_set=(val_x,val_y), eval_metric="auc", callbacks=cb_list)
        best_ntrees[0] = model.best_iteration_
    elif model_name == "XGB_GBT":
        # objective
        # regession : "reg:absoluteerror", "reg:squarederror"
        # classification - binary : "binary:logistic"
        # classification - multicalss :"multi:softmax" (num_class=n)
        # ranking : "rank:ndcg"

        # metric
        # regession : "mae", "rmse"
        # classification - binary : "logloss", "error@t" (t=threshold), "auc"
        # classification - multicalss : "mlogloss", "merror"
        # ranking : "ndcg", "map"

        tuning_params = {
            "learning_rate": trial.suggest_categorical("learning_rate", [1e-2, 5e-3, 1e-3]),
            "max_depth": trial.suggest_categorical("max_depth", [4, 5, 6, 7, 8]),
            "subsample": trial.suggest_float("subsample", 0.5, 0.8, step=0.1),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 0.8, step=0.1),
            "reg_lambda": trial.suggest_categorical("reg_lambda", list(np.linspace(1e-3, 1.0, num=150, endpoint=False)) + list(np.linspace(1.0, 1e+2, num=50, endpoint=True))),
            "min_child_weight": trial.suggest_categorical("min_child_weight", list(np.linspace(1e-3, 1.0, num=150, endpoint=False)) + list(np.linspace(1.0, 1e+2, num=50, endpoint=True))),
            "gamma": trial.suggest_categorical("gamma", list(np.linspace(1e-3, 1.0, num=150, endpoint=False)) + list(np.linspace(1.0, 1e+2, num=50, endpoint=True))),
            # for binary
            "scale_pos_weight": trial.suggest_categorical("scale_pos_weight", list(np.linspace(1e-3, 1.0, num=150, endpoint=False)) + list(np.linspace(1.0, 1e+2, num=50, endpoint=True))),
        }
        model = xgb.XGBClassifier(booster="gbtree", objective="binary:logistic",
                            n_estimators=ntrees, tree_method="gpu_hist",
                            random_state=fold, verbosity=0, **tuning_params)
        model.fit(train_x, train_y,
                  eval_set=[(val_x, val_y)], eval_metric="auc",
                  early_stopping_rounds=int(ntrees * 0.2), verbose=False)
        best_ntrees[0] = model.best_iteration
    elif model_name == "CAT_GBM":
#         CatBoostRanker, Pool, MetricVisualizer
        # objective
        # regession : "MAE", "RMSE"
        # classification - binary : "Logloss"
        # classification - multicalss :"MultiClass"
        # ranking :
        # past -> slow, low quality -> high quality
        # RMSE
        # QueryRMSE
        # PairLogit
        # PairLogitPairwise
        # YetiRank
        # YetiRankPairwise

        # metric
        # regession : "MAE", "RMSE", "R2"
        # classification - binary : "Logloss", "Accuracy", "AUC", "F1"
        # classification - multicalss : "MultiClass", "Accuracy", "TotalF1" (average=Weighted,Macro,Micro)
        # ranking : "PairLogit", "YetiRank", "NDCG", "MAP"

        tuning_params = {
            "learning_rate": trial.suggest_categorical("learning_rate", [5e-3]),
            "max_depth": trial.suggest_categorical("max_depth", [4, 6, 8]),
            "bagging_temperature": trial.suggest_categorical("bagging_temperature", list(np.linspace(1e-3, 1.0, num=150, endpoint=False)) + list(np.linspace(1.0, 1e+2, num=50, endpoint=True))),
            # rsm = colsample_bylevel (not supported for GPU)
            # "rsm": trial.suggest_float("rsm", 0.5, 0.8, step=0.1)
            "random_strength": trial.suggest_categorical("random_strength", [0.01, 0.05, 0.1, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0]),
            "reg_lambda": trial.suggest_categorical("reg_lambda", list(np.linspace(1e-3, 1.0, num=150, endpoint=False)) + list(np.linspace(1.0, 1e+2, num=50, endpoint=True))),
            "min_child_samples": trial.suggest_float("min_child_samples", 1, 51, step=2),
            # for binary
            # "scale_pos_weight": trial.suggest_categorical("scale_pos_weight", list(np.linspace(1e-3, 1.0, num=150, endpoint=False)) + list(np.linspace(1.0, 1e+2, num=50, endpoint=True))),
        }

#         model = cat.CatBoostClassifier(boosting_type="Plain", loss_function="YetiRank", eval_metric="Logloss",
#                                     n_estimators=ntrees, task_type="GPU", bootstrap_type="Bayesian",
#                                     verbose=False, random_state=fold, **tuning_params)
        model = cat.CatBoostRanker(boosting_type="Plain", loss_function="QueryRMSE", eval_metric="QueryRMSE",
                            n_estimators=ntrees, task_type="GPU", bootstrap_type="Bayesian",
                            verbose=False, random_state=fold, **tuning_params)
    
        train_pool = cat.Pool(
            data=train_x,
            label=train_y,
            group_id=train_groups
        )

        val_pool = cat.Pool(
            data=val_x,
            label=val_y,
            group_id=val_groups
        )

        model.fit(train_pool, cat_features=categoIdx,
                eval_set=val_pool, early_stopping_rounds=int(ntrees * 0.2), use_best_model=True,
                verbose=False)
        best_ntrees[0] = model.best_iteration_
    elif model_name == "CAT_ORD":
        # objective
        # regession : "MAE", "RMSE"
        # classification - binary : "Logloss"
        # classification - multicalss :"MultiClass"
        # ranking : "PairLogit", "YetiRank"

        # metric
        # regession : "MAE", "RMSE", "R2"
        # classification - binary : "Logloss", "Accuracy", "AUC", "F1"
        # classification - multicalss : "MultiClass", "Accuracy", "TotalF1" (average=Weighted,Macro,Micro)
        # ranking : "PairLogit", "YetiRank", "NDCG", "MAP"

        tuning_params = {
            "learning_rate": trial.suggest_categorical("learning_rate", [5e-3]),
            "max_depth": trial.suggest_categorical("max_depth", [4, 6, 8]),
            # "bagging_temperature": trial.suggest_categorical("bagging_temperature", list(np.linspace(1e-3, 1.0, num=75, endpoint=False)) + list(np.linspace(1.0, 1e+2, num=25, endpoint=True))),
            # rsm = colsample_bylevel (not supported for GPU)
            # "rsm": trial.suggest_float("rsm", 0.5, 0.8, step=0.1),
            "random_strength": trial.suggest_categorical("random_strength", [0.01, 0.1, 1.0, 2.0, 3.0]),
            "reg_lambda": trial.suggest_categorical("reg_lambda", list(np.linspace(1e-3, 1.0, num=150, endpoint=False)) + list(np.linspace(1.0, 1e+2, num=50, endpoint=True))),
            # "min_child_samples": trial.suggest_float("min_child_samples", 1, 51, step=2),
            # for binary
            # "scale_pos_weight": trial.suggest_categorical("scale_pos_weight", list(np.linspace(1e-3, 1.0, num=150, endpoint=False)) + list(np.linspace(1.0, 1e+2, num=50, endpoint=True))),
        }

        model = cat.CatBoostClassifier(boosting_type="Ordered", loss_function="Logloss", eval_metric="Logloss",
                                    n_estimators=ntrees, task_type="GPU", bootstrap_type="Bayesian",
                                    verbose=False, random_state=fold, **tuning_params)
        model.fit(train_x, train_y, cat_features=categoIdx,
                eval_set=[(val_x, val_y)], early_stopping_rounds=int(ntrees * 0.2), use_best_model=True,
                verbose=False)
        best_ntrees[0] = model.best_iteration_
    else:
        print("unknown")
        return -1
    
    pred = df_full.loc[ids[val_idx]].copy()
    pred.loc[(pred["cell_type"] == 1.0), "rank"] = model.predict(val_pool)[(pred["cell_type"] == 1.0)]
    optuna_score = kendall_tau(df_orders.loc[ids[val_idx]].sort_index(), convert_rank_ids(pred.sort_values(["id", "rank"], ascending=[True, False])))
    
#     # classification
#     pred = model.predict_proba(val_x)
#     optuna_score = metrics.f1_score(val_y.tolist(), [1.0 if i > threshold else 0.0 for i in pred[:,1]])
    
#     # ranking
#     pred = model.predict(val_x)
#     optuna_score = metrics.mean_squared_error(val_y.tolist(), pred)

    if optuna_score > output_container["score"]:
        print("found the best !")
        if best_ntrees[0] is not None:
            print("number of trees :", best_ntrees)
        output_container["model"] = model
        output_container["pred"] = pred
        output_container["score"] = optuna_score

    # trial.report(optuna_score, trial.number)
    # if trial.should_prune():
    #     raise optuna.TrialPruned()

    return optuna_score

In [ ]:
def do_fold_training(fold, train_idx, val_idx, sample_weight=None, finetuning=False):
    tmp_time = time()
    print("\n===== Fold", fold, "=====\n")
    
    train_x = df_full.loc[(df_full.index.get_level_values(0).isin(ids[val_idx]) & (df_full["cell_type"] == 1.0).values)]
    train_groups = train_x.index.get_level_values(0).to_list()
    train_y = train_x["rank"].copy()
    train_x = df_full_fv[(df_full.index.get_level_values(0).isin(ids[val_idx]) & (df_full["cell_type"] == 1.0).values)]
    
    val_x = df_full.loc[ids[val_idx]]
    val_groups = val_x.index.get_level_values(0).to_list()
    val_y = val_x["rank"].copy()
    val_x = df_full_fv[findIdx(df_full.index.get_level_values(0), ids[val_idx])]
    
    output_container = {"model": None, "pred": None, "score": -np.inf}
    optuna_direction = 'maximize'
    optuna_trials = 5
    optuna_timout = int(6 * 3600 / kfolds_spliter.get_n_splits())
    optuna_study = create_study(direction=optuna_direction, sampler=TPESampler())
    
    best_ntrees = [0]
    optuna_study.optimize(
        lambda trial: optuna_objective_function(
            trial, fold, train_x, train_y, train_groups, val_x, val_y, val_groups, categoIdx=None, model_name="CAT_GBM", output_container=output_container,
            ntrees=ntrees, eta=eta, best_ntrees=best_ntrees
        ),
        n_jobs=1, n_trials=optuna_trials, timeout=optuna_timout
    )
    
    model_list.append(output_container["model"])
    params_list.append(optuna_study.best_params)
    if best_ntrees[0] is not None:
        params_list[-1]["ntrees"] = best_ntrees[0]
    print("fold", fold, "best params :", params_list[-1])
    
    val_pred.loc[ids[val_idx], "rank"] = output_container["pred"]["rank"]
    fold_metric.append(output_container["score"])
    print("fold", fold, "score :", fold_metric[-1])
    
    test_x = df_test.copy()
    test_groups = test_x.index.get_level_values(0).to_list()
    test_y = None
    test_x = df_test_fv[:]
    
    test_pool = cat.Pool(
        data=test_x,
        label=test_y,
        group_id=test_groups
    )
    
    test_pred.loc[(test_pred["cell_type"] == 1.0), "rank"] += (model_list[-1].predict(test_pool) / n_folds)[(test_pred["cell_type"] == 1.0)]

In [ ]:
# learning parameter setting
ntrees = 5000
eta = 5e-3

# ntrees = 100
# eta = 1e-2

# for group fold split
ids = df_full.index.unique('id')
df_ancestors = pd.read_csv(data_dir / 'train_ancestors.csv', index_col='id')
ancestors = df_ancestors.loc[ids, 'ancestor_id']

# stratified sampling by each length of code blocks
nb_length_spliter = KBinsDiscretizer(n_bins=10, strategy="quantile", encode="ordinal")
strat_y = nb_length_spliter.fit_transform(df_full.groupby('id').size().to_frame()).flatten().astype("int32")

In [ ]:
model_list = []
history_list = []
params_list = []
fold_metric = []

val_pred = df_full[["cell_type", "rank"]].copy()
val_pred.loc[val_pred["cell_type"]==1.0, "rank"] = 0
test_pred = df_test[["cell_type", "rank"]].copy()
test_pred.loc[test_pred["cell_type"]==1.0, "rank"] = 0

n_folds = 5
kfolds_spliter = StratifiedGroupKFold(n_folds, shuffle=True, random_state=42)

start_time_training = time()
# fold training
for fold, (train_idx, val_idx) in enumerate(kfolds_spliter.split(range(len(ids)), y=strat_y, groups=ancestors)):
    start_mem = memory_usage()   
    do_fold_training(fold, train_idx, val_idx, None, finetuning=False)
    gc.collect()
    end_mem = memory_usage()
    print("@Memory leaked :", end_mem - start_mem, "\n")
end_time_training = time()

In [ ]:
for idx, value in enumerate(fold_metric):
    print("Fold", idx, "score :", value)

In [ ]:
print("Average score :", np.mean(fold_metric))

# Submission

In [ ]:
# sample submission on markdown
test_pred.loc[(test_pred["cell_type"] == 1.0), "rank"].head(20)

In [ ]:
test_pred.loc[(test_pred["cell_type"] == 1.0), "rank"].hist()

In [ ]:
y_submit = (
    convert_rank_ids(test_pred.sort_values(["id", "rank"], ascending=[True, False]))
    .apply(' '.join)  # list of ids -> string of ids
    .rename_axis('id')
    .rename('cell_order')
)
y_submit

In [ ]:
y_submit.to_csv('submission.csv')